In [1]:
#Importing Necessary Libraries
import json 
import os
import csv
import pickle
import sqlite3
import pandas as pd
import seaborn as sns
from numba import jit
import scipy
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
%matplotlib inline

## Database Operation Functions


In [ ]:
def connect(path):
    con=sqlite3.connect(path)
    return con


def createQuery(query,connection):
    try:
        connection.cursor().execute(query)
        print('Table Created')
    except Exception as e:
        print('Exception',e)
        


#Function to perform get queries
def getQuery(query,connection):
    try:
        result=connection.cursor().execute(query)
        print('Query Executed')
        return result.fetchall()
    except Exception as e:
        print('Exception',e)

        
#Function to convert query(which comes in as a list of tuples to DataFrame)
def toDf(res,cols):
    try:
        return pd.DataFrame(res,columns=cols)
    except Exception as e:
        print(f'The DataFrame cannot be created because of this error: {e}')

        
#Generating Schema for Tables
        
def generateSchema(tableName,df):
    schema=f'create table if not exists {tableName}('
    cols=df.columns
    dataType={'int64':'int','float64':'int','object':'varchar(100)','bool':'boolean'}
    for i in range(len(cols)):
        name=''.join(cols[i].split(' '))
        if name.startswith('Date'):
            definition=f'{name} DATE,'
            schema+=definition
            continue
        if name.startswith('Yea'):
            definition=f'{name} YEAR,'
            schema+=definition
            continue
            
        if i!=len(cols)-1:
            definition=f'{name} {dataType[str(df[cols[i]].dtype)]}, '
            schema+=definition
        else:
            definition=f'{name} {dataType[str(df[cols[i]].dtype)]})'
            schema+=definition
    return schema

#Function Transformed

def getQuery(query,connection,cols):
    try:
        result=connection.cursor().execute(query)
        print('Query Executed')
        return toDf(result.fetchall(),cols)
    except Exception as e:
        print('Exception',e)


## Creation of Database

In [ ]:
#We now change our connection to the newly created database.
connection=connect('../input/yelpdb/database.db')
# We perform an inner join on the buisness and reviews table to get all the users that rated a buisness
buisnessJoined=getQuery('select a.business_id, b.user_id, b.stars from business a INNER JOIN reviews b where a.business_id=b.business_id',connection,['business_id','user_id','stars'])

In [ ]:
user_index={u:i for i,u in enumerate(buisnessJoined.user_id.unique())}
busi_index={u:i for i,u in enumerate(buisnessJoined.business_id.unique())}
temp_1=buisnessJoined.user_id.apply(lambda x:user_index[x])
temp_2=buisnessJoined.business_id.apply(lambda x:busi_index[x])
buisnessJoined.drop(['business_id','user_id'],axis=1,inplace=True)
buisnessJoined=pd.concat([buisnessJoined,temp_1,temp_2],axis=1)
buisnessJoined.drop('Unnamed: 0',axis=1,inplace=True)
buisnessJoined=buisnessJoined.astype('int32')


In [94]:
# To be used in Kaggle
buisnessJoined=pd.read_csv('../input/baseline-model/buisnessJoined.csv')
buisnessJoined.drop('Unnamed: 0',axis=1,inplace=True)

In [95]:
#Getting top users by the number of ratings given
user_idx=buisnessJoined.groupby(['user_id']).count().sort_values('stars',ascending=False).index.values
buisnessJoined=buisnessJoined.iloc[user_idx].reset_index(drop=True)

In [98]:
#Finding out the average rating of every individual business/restaurant
business_stars=buisnessJoined.groupby(['business_id']).mean('stars')['stars']
#Finding out the average rating given by a every user to every business
user_stars =buisnessJoined.groupby(['user_id']).mean('stars')['stars']
#Average over the whole sample dataset
overall_avg_stars=buisnessJoined.stars.sum()/buisnessJoined.shape[0]

In [99]:
#User bias 'theta' and Item bias 'gamma'
theta=np.array(user_stars-overall_avg_stars)
gamma=np.array(business_stars-overall_avg_stars)


In [107]:
#Taking only 500 users
users_taken=5000
Y={}
for u in range(users_taken):
    temp=[]
    for m in range(len(gamma)):
        temp.append(overall_avg_stars-theta[u]+gamma[m])
    Y[u]=temp


In [108]:
#Creating a Dataframe for convenient viewing
Y_df=pd.DataFrame(Y)
Y_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,4.256026,3.006026,3.779282,2.756026,4.217564,3.381026,2.756026,2.756026,2.756026,4.506026,...,3.422693,6.756026,2.756026,3.956026,4.033804,4.066837,4.346935,6.256026,2.756026,4.033804
1,4.449824,3.199824,3.973080,2.949824,4.411363,3.574824,2.949824,2.949824,2.949824,4.699824,...,3.616491,6.949824,2.949824,4.149824,4.227602,4.260635,4.540734,6.449824,2.949824,4.227602
2,3.941740,2.691740,3.464996,2.441740,3.903279,3.066740,2.441740,2.441740,2.441740,4.191740,...,3.108407,6.441740,2.441740,3.641740,3.719518,3.752551,4.032649,5.941740,2.441740,3.719518
3,4.983299,3.733299,4.506555,3.483299,4.944837,4.108299,3.483299,3.483299,3.483299,5.233299,...,4.149965,7.483299,3.483299,4.683299,4.761076,4.794110,5.074208,6.983299,3.483299,4.761076
4,3.856026,2.606026,3.379282,2.356026,3.817564,2.981026,2.356026,2.356026,2.356026,4.106026,...,3.022693,6.356026,2.356026,3.556026,3.633804,3.666837,3.946935,5.856026,2.356026,3.633804


In [110]:
# Forming the sparsity matrix
ratings=np.zeros((users_taken, n_items))

for row in buisnessJoined.itertuples():
    if row[2]<users_taken:
      ratings[row[2], row[3]] = row[1]
      


In [112]:
#Getting the non-zero values
idx=np.where(ratings.flatten()!=0)

#Calculating MSE
mse=mean_squared_error(ratings.flatten()[idx],Y_df.values.flatten()[idx])
print('MSE of the most basic recommendation system =',np.round(mse,2))

MSE of the most basic recommendation system = 3.02
